# Action Rules

Target: Evaluation (1 - best, 5 - worst) 

Research Questions:
1. How to change Evaluation from bad (4, 5) to good (1, 2)?
2. How to change an average evaluation (3) to the best (1).
3. How to change Evaluation from bad (4, 5) to good (1, 2) per fixed area?

## Load data
The M17+ dataset is used in these experiments. The dataset consists of 7710 articles from Czech research institutions that were graded by a jury of experts in a program subsidized by the government of the Czech Republic. The target variable in this dataset is an ordinal variable ranging from 1 to 5, with 1 meaning that the article is of world-class quality and impact, while 5 means that the article is of mediocre quality and impact.

In [1]:
import pandas as pd
import numpy as np
import json
pd.options.display.max_columns = None

data = pd.read_csv("m17_llm_features.csv")
data

,title,abstract,science_field,evaluation,novelty,rigor,grammar,replicability,accessibility,mathematics,computer_and_information_sciences,physical_sciences,chemical_sciences,earth_and_related_environmental_sciences,biological_sciences,other_natural_sciences,civil_engineering,electrical_engineering,mechanical_engineering,chemical_engineering,materials_engineering,environmental_engineering,evironmental_biotechnology,industrial_biotechnology,nano-technology,other_engineering_and_technologies,basic_medicine,clinical_medicine,health_sciences,medical_biotechnology,other_medical_sciences,agriculture_forestry_and_fisheries,animal_and_dairy_science,veterinary_science,agricultural_biotechnology,other_agricultural_sciences,psychology,economics_and_business,education,sociology,law,political_science,social_and_economic_geography,media_and_communications,other_social_sciences,history_and_archaeology,languages_and_literature,philosophy_ethics_and_religion,arts,other_humanities_and_the_arts,applied_research,conclusive_research,correlational_research,descriptive_research,ethnographic_research,experiment,exploratory_research,focus_groups,interviews,mixed_methods,observation,qualitative_research,quantitative_research,questionnaires,secondary_research,survey_methodology
0,The making of a socialist underworld: people o...,This text depicts the mechanisms that created ...,60101.0,1,medium,high,0,0,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Selective Translation Complex Profiling Reveal...,Translational control targeting the initiation...,10608.0,1,high,high,0,1,medium,0,1,1,1,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,Layered Black Phosphorus: Strongly Anisotropic...,"Layered elemental materials, such as black pho...",0.0,1,high,high,1,1,medium,0,1,1,1,0,0,1,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Geochemical Modelling of Igneous Processes - P...,The aim of this book is to unlock the power of...,0.0,1,medium,medium,0,0,medium,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Plant diversity increases with the strength of...,Theory predicts that higher biodiversity in th...,40102.0,1,high,high,0,1,medium,0,0,0,0,1,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,1,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,Evaluation of Aberration-corrected Optical Sec...,The introduction of spherical-aberration corre...,10302.0,5,medium,high,0,1,medium,0,1,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1996,Interfacial interaction between CMS layer and ...,The claim that quality of membrane fabrication...,20402.0,5,high,high,0,1,medium,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,0,1,1,1,0,1,1,0,1,1,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1997,Rating and Review of a Film Inspired by Africa...,People watch films for several reasons. Films ...,60404.0,5,medium,medium,1,0,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1998,Gwyscan,"Scanning probe microscopy support library, inc...",NaN,5,medium,medium,0,0,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Prepare Targets
The target variable is converted into a binary format.

In [2]:
data["evaluation1"] = data['evaluation'].apply(lambda x: 'good' if int(x) in [1, 2] else ('bad' if int(x) in [4, 5] else np.nan))
data["evaluation2"] = data['evaluation'].apply(lambda x: 'average' if int(x) in [3] else ('best' if int(x) in [1] else np.nan))

## Drop unused columns
Non-categorical attributes cannot be utilized for the discovery of action rules.

In [4]:
stop = ['title', 'abstract', 'science_field', 'evaluation']

In [5]:
data = data.drop(columns=stop)

## Create groups of features
The features need to be grouped, and within these groups, we recommend the changes. The package is designed to expect a single value per attribute in each transaction. However, in this case, since multiple values exist, in the action-rules package, a fit_onehot method was developed to handle multiple values for an attribute.

In [6]:
research = ['quantitative_research', 
            'experiment', 
            'exploratory_research', 
            'qualitative_research', 
            'mixed_methods', 
            'observation', 
            'applied_research', 
            'descriptive_research', 
            'focus_groups', 
            'survey_methodology', 
            'correlational_research', 
            'interviews', 
            'questionnaires', 
            'secondary_research', 
            'conclusive_research', 
            'ethnographic_research']
area = ['mathematics',
        'computer_and_information_sciences',
        'physical_sciences',
        'chemical_sciences',
        'earth_and_related_environmental_sciences',
        'biological_sciences',
        'other_natural_sciences',
        'civil_engineering',
        'electrical_engineering',
        'mechanical_engineering',
        'chemical_engineering',
        'materials_engineering',
        'environmental_engineering',
        'evironmental_biotechnology',
        'industrial_biotechnology',
        'nano-technology',
        'other_engineering_and_technologies',
        'basic_medicine',
        'clinical_medicine',
        'health_sciences',
        'medical_biotechnology',
        'other_medical_sciences',
        'agriculture_forestry_and_fisheries',
        'animal_and_dairy_science',
        'veterinary_science',
        'agricultural_biotechnology',
        'other_agricultural_sciences',
        'psychology',
        'economics_and_business',
        'education',
        'sociology',
        'law',
        'political_science',
        'social_and_economic_geography',
        'media_and_communications',
        'other_social_sciences',
        'history_and_archaeology',
        'languages_and_literature',
        'philosophy_ethics_and_religion',
        'arts',
        'other_humanities_and_the_arts']

## Onehot encoding
One-hot encoding is a method used to convert categorical variables into a binary vector representation. In this encoding scheme, each category is represented by a vector in which one element is set to 1 (indicating the presence of that category) and all other elements are set to 0. This approach allows categorical data to be used in machine learning models that require numerical input.

In [7]:
labels_to_onehot = ['novelty', 'rigor', 'grammar',	'replicability', 'accessibility']
data[labels_to_onehot] = data[labels_to_onehot].astype('string')
data_flexible = pd.get_dummies( 
        data[labels_to_onehot], sparse=False, prefix_sep='_'
    )
target_cols = ['evaluation1', 'evaluation2']
data_target = pd.get_dummies(data[target_cols], sparse=False, prefix_sep='_')
rest_data = data[research + area]

In [8]:
df = pd.concat([data_flexible, rest_data, data_target], axis=1)
df = df.astype('bool')
df

,novelty_high,novelty_low,novelty_medium,rigor_high,rigor_low,rigor_medium,grammar_0,grammar_1,replicability_0,replicability_1,accessibility_high,accessibility_low,accessibility_medium,quantitative_research,experiment,exploratory_research,qualitative_research,mixed_methods,observation,applied_research,descriptive_research,focus_groups,survey_methodology,correlational_research,interviews,questionnaires,secondary_research,conclusive_research,ethnographic_research,mathematics,computer_and_information_sciences,physical_sciences,chemical_sciences,earth_and_related_environmental_sciences,biological_sciences,other_natural_sciences,civil_engineering,electrical_engineering,mechanical_engineering,chemical_engineering,materials_engineering,environmental_engineering,evironmental_biotechnology,industrial_biotechnology,nano-technology,other_engineering_and_technologies,basic_medicine,clinical_medicine,health_sciences,medical_biotechnology,other_medical_sciences,agriculture_forestry_and_fisheries,animal_and_dairy_science,veterinary_science,agricultural_biotechnology,other_agricultural_sciences,psychology,economics_and_business,education,sociology,law,political_science,social_and_economic_geography,media_and_communications,other_social_sciences,history_and_archaeology,languages_and_literature,philosophy_ethics_and_religion,arts,other_humanities_and_the_arts,evaluation1_bad,evaluation1_good,evaluation2_average,evaluation2_best
0,False,False,True,True,False,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,True,True,False,False,True,False,True,False,True
1,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,True,True,True,True,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True
2,True,False,False,True,False,False,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,True,False,True,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True
3,False,False,True,False,False,True,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True
4,True,False,False,True,False,False,True,False,False,True,False,False,True,True,False,True,True,True,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,True,True,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,True,True,True,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,False,False,True,True,False,False,True,False,False,True,False,False,True,False,False,False,False,False,False,F

In [9]:
novelty = ['novelty_high', 'novelty_low', 'novelty_medium']
rigor = ['rigor_high', 'rigor_low', 'rigor_medium']
grammar = ['grammar_0', 'grammar_1']
replicability = ['replicability_0', 'replicability_1']
accessibility = ['accessibility_high', 'accessibility_low', 'accessibility_medium']

### Prepare stable and flexible attributes
For action rule mining, attributes are divided into two categories: stable attributes, which remain unchanged and serve as fixed conditions in the rules, and flexible attributes, where changes are pursued to achieve the desired outcome. The attribute area is classified as stable, while novelty, rigor, grammar, replicability, accessibility, and research are categorized as flexible attributes.

In [10]:
stable = {'area': area}
flexible = {'novelty': novelty, 'rigor': rigor, 'grammar': grammar, 'replicability': replicability, 'accessibility': accessibility, 'research': research}
target1 = {'evaluation': ['evaluation1_bad', 'evaluation1_good']}
target2 = {'evaluation': ['evaluation2_average', 'evaluation2_best']}

## Experiment 1: Bad (4,5) to Good (1,2)

### ActionRules Miner Initialization

In [11]:
from action_rules import ActionRules
ar = ActionRules(
    min_stable_attributes=0,
    min_flexible_attributes=1,
    min_undesired_support=60,
    min_undesired_confidence=0.7,
    min_desired_support=60,
    min_desired_confidence=0.7,
    verbose=False)

### Model Training

In [12]:
ar.fit_onehot(
    data=df,           
    stable_attributes=stable,
    flexible_attributes=flexible,
    target=target1,
    target_undesired_state='evaluation1_bad',
    target_desired_state='evaluation1_good',
    use_sparse_matrix=False,
    use_gpu=True)

### Print

In [13]:
r = ar.get_rules().get_ar_notation()
print('Found: '+str(len(r))+' action rules')
print()
for action_rule in r:
    print(action_rule)
    print()

Found: 42 action rules

[(rigor: rigor_medium → rigor_high)] ⇒ [evaluation: evaluation1_bad → evaluation1_good], support of undesired part: 561, confidence of undesired part: 0.7110266159695817, support of desired part: 570, confidence of desired part: 0.7187894073139974, uplift: 0.169562421185372

[(novelty: novelty_medium → novelty_high) ∧ (rigor: rigor_medium → rigor_high)] ⇒ [evaluation: evaluation1_bad → evaluation1_good], support of undesired part: 390, confidence of undesired part: 0.7208872458410351, support of desired part: 262, confidence of desired part: 0.8534201954397395, uplift: 0.1553501628664495

[(novelty: novelty_low → novelty_high) ∧ (grammar: grammar_1 → grammar_0)] ⇒ [evaluation: evaluation1_bad → evaluation1_good], support of undesired part: 106, confidence of undesired part: 0.7969924812030075, support of desired part: 187, confidence of desired part: 0.8577981651376146, uplift: 0.043543577981651364

[(novelty: novelty_low → novelty_high) ∧ (grammar*: grammar_1)]

### Dominant Action Rules
Dominant action rules code filters out redundant rules, ensuring that only the most meaningful and relevant rules are retained.

In [14]:
def get_important_rules(json_rules, notation_rules):
    important_rules = []
    
    def create_stable_set(values):
        stable_tuples = [(d["attribute"], d["value"]) for d in values]
        return set(stable_tuples)
    
    def create_flexible_set(values):
        flexible_tuples = [(d["attribute"], d["undesired"], d["desired"]) for d in values]
        return set(flexible_tuples)
    
    candidate_set = create_stable_set(json_rules[0]['stable']) | create_flexible_set(json_rules[0]['flexible'])
    candidate = json_rules[0]
    candidate['to_delete'] = False
    candidate['candidate_set'] = candidate_set
    candidate['rule_index'] = 0
    important_rules.append(candidate)
    
    i = 0
    for candidate in json_rules[1:]:
        i += 1
        candidate_set = create_stable_set(candidate['stable']) | create_flexible_set(candidate['flexible'])
        is_add_rule = True
        for important_rule in important_rules:
            if important_rule['to_delete']:
                continue
            #if important_rule['candidate_set'] < candidate_set and important_rule['uplift'] < candidate['uplift']:
                #continue
            if important_rule['candidate_set'] < candidate_set and important_rule['uplift'] >= candidate['uplift']:
                is_add_rule = False
            elif important_rule['candidate_set'] > candidate_set and important_rule['uplift'] <= candidate['uplift']:
                important_rule['to_delete'] = True
        if is_add_rule:
            candidate['to_delete'] = False
            candidate['candidate_set'] = candidate_set
            candidate['rule_index'] = i
            important_rules.append(candidate)
                
    important_rules = [rule for rule in important_rules if not rule['to_delete']]
    sorted_indices = sorted(important_rules, key=lambda x: x["uplift"], reverse=True)
    important_rules_indices = [rule['rule_index'] for rule in sorted_indices]
    for i in important_rules_indices:
        print(notation_rules[i])
        print()  



In [15]:
candidate_rules = json.loads(ar.get_rules().get_export_notation())
get_important_rules(candidate_rules, r)

[(rigor: rigor_medium → rigor_high) ∧ (replicability: replicability_0 → replicability_1)] ⇒ [evaluation: evaluation1_bad → evaluation1_good], support of undesired part: 480, confidence of undesired part: 0.7384615384615385, support of desired part: 398, confidence of desired part: 0.7834645669291339, uplift: 0.16962598425196854

[(rigor: rigor_medium → rigor_high)] ⇒ [evaluation: evaluation1_bad → evaluation1_good], support of undesired part: 561, confidence of undesired part: 0.7110266159695817, support of desired part: 570, confidence of desired part: 0.7187894073139974, uplift: 0.169562421185372

[(grammar: grammar_1 → grammar_0) ∧ (replicability: replicability_0 → replicability_1)] ⇒ [evaluation: evaluation1_bad → evaluation1_good], support of undesired part: 344, confidence of undesired part: 0.7413793103448276, support of desired part: 337, confidence of desired part: 0.7439293598233996, uplift: 0.11259161147902871

[(novelty: novelty_low → novelty_high) ∧ (replicability: replica

## Experiment 2: Average (3) to Best (1)

### ActionRules Miner Initialization

In [18]:
from action_rules import ActionRules
ar2 = ActionRules(
    min_stable_attributes=0,
    min_flexible_attributes=1,
    min_undesired_support=30,
    min_undesired_confidence=0.7,
    min_desired_support=40,
    min_desired_confidence=0.7,
    verbose=False)

### Model Training

In [19]:
ar2.fit_onehot(
    data=df,           
    stable_attributes=stable,
    flexible_attributes=flexible,
    target=target2,
    target_undesired_state='evaluation2_average',
    target_desired_state='evaluation2_best',
    use_sparse_matrix=False,
    use_gpu=True)

### Print

In [20]:
r2 = ar2.get_rules().get_ar_notation()
print('Found: '+str(len(r2))+' action rules')
print()
for action_rule in r2:
    print(action_rule)
    print()

Found: 22 action rules

[(novelty: novelty_low → novelty_high) ∧ (rigor: rigor_medium → rigor_high)] ⇒ [evaluation: evaluation2_average → evaluation2_best], support of undesired part: 68, confidence of undesired part: 0.723404255319149, support of desired part: 164, confidence of desired part: 0.7522935779816514, uplift: 0.02235779816513762

[(novelty: novelty_medium → novelty_high) ∧ (rigor: rigor_medium → rigor_high)] ⇒ [evaluation: evaluation2_average → evaluation2_best], support of undesired part: 137, confidence of undesired part: 0.7445652173913043, support of desired part: 164, confidence of desired part: 0.7522935779816514, uplift: 0.04571100917431192

[(novelty: novelty_low → novelty_high) ∧ (grammar: grammar_1 → grammar_0)] ⇒ [evaluation: evaluation2_average → evaluation2_best], support of undesired part: 34, confidence of undesired part: 0.8292682926829268, support of desired part: 122, confidence of desired part: 0.7770700636942676, uplift: 0.012429936305732484

[(novelty: 

### Dominant Action Rules

In [21]:
candidate_rules2 = json.loads(ar2.get_rules().get_export_notation())
get_important_rules(candidate_rules2, r2)

[(novelty: novelty_medium → novelty_high) ∧ (rigor: rigor_medium → rigor_high)] ⇒ [evaluation: evaluation2_average → evaluation2_best], support of undesired part: 137, confidence of undesired part: 0.7445652173913043, support of desired part: 164, confidence of desired part: 0.7522935779816514, uplift: 0.04571100917431192

[(novelty: novelty_medium → novelty_high) ∧ (grammar: grammar_1 → grammar_0)] ⇒ [evaluation: evaluation2_average → evaluation2_best], support of undesired part: 105, confidence of undesired part: 0.7, support of desired part: 122, confidence of desired part: 0.7770700636942676, uplift: 0.035780254777070065

[(novelty: novelty_low → novelty_high) ∧ (rigor: rigor_medium → rigor_high)] ⇒ [evaluation: evaluation2_average → evaluation2_best], support of undesired part: 68, confidence of undesired part: 0.723404255319149, support of desired part: 164, confidence of desired part: 0.7522935779816514, uplift: 0.02235779816513762

[(novelty: novelty_low → novelty_high) ∧ (repl

## Experiment 3: Bad (4,5) to Good (1,2) per area

In [24]:
rules_area = {}
for area_value in area:
    stable_attr = {'area': [area_value]}
    arules = ActionRules(
        min_stable_attributes=1,
        min_flexible_attributes=1,
        min_undesired_support=35,
        min_undesired_confidence=0.7,
        min_desired_support=35,
        min_desired_confidence=0.7,
        verbose=False)
    arules.fit_onehot(
        data=df,           
        stable_attributes=stable_attr,
        flexible_attributes=flexible,
        target=target1,
        target_undesired_state='evaluation1_bad',
        target_desired_state='evaluation1_good',
        use_sparse_matrix=False,
        use_gpu=True)
    rules_area[area_value] = [arules.get_rules().get_ar_notation(), arules.get_rules().get_export_notation()]

In [25]:
no_rules = []
for stable, rules in rules_area.items():
    if len(rules[0]) > 0:
        print(f"The area: {stable}")
        print(f"Number of rules: {len(rules[0])}")
        rules_list = json.loads(rules[1])
        max_uplift = max(rules_list, key=lambda x: x["uplift"])
        max_uplift_id = rules_list.index(max_uplift)
        # print(f"The ID of the rule with the highest uplift is: {max_uplift_id}")
        print('Rule with the highest uplift:')
        print(rules[0][max_uplift_id])
        print('--------------')
    else:
        no_rules.append(stable)

The area: computer_and_information_sciences
Number of rules: 4
Rule with the highest uplift:
[(area: computer_and_information_sciences) ∧ (novelty: novelty_medium → novelty_high) ∧ (rigor: rigor_medium → rigor_high)] ⇒ [evaluation: evaluation1_bad → evaluation1_good], support of undesired part: 50, confidence of undesired part: 0.7142857142857143, support of desired part: 204, confidence of desired part: 0.85, uplift: 0.01975
--------------
The area: electrical_engineering
Number of rules: 2
Rule with the highest uplift:
[(area: electrical_engineering) ∧ (rigor: rigor_medium → rigor_high) ∧ (accessibility*: accessibility_medium)] ⇒ [evaluation: evaluation1_bad → evaluation1_good], support of undesired part: 36, confidence of undesired part: 0.75, support of desired part: 204, confidence of desired part: 0.7285714285714285, uplift: 0.011485714285714283
--------------
The area: mechanical_engineering
Number of rules: 5
Rule with the highest uplift:
[(area: mechanical_engineering) ∧ (nove

Areas without rules

In [26]:
no_rules

['mathematics',
 'physical_sciences',
 'chemical_sciences',
 'earth_and_related_environmental_sciences',
 'biological_sciences',
 'other_natural_sciences',
 'civil_engineering',
 'environmental_engineering',
 'evironmental_biotechnology',
 'industrial_biotechnology',
 'nano-technology',
 'other_engineering_and_technologies',
 'basic_medicine',
 'clinical_medicine',
 'health_sciences',
 'medical_biotechnology',
 'other_medical_sciences',
 'agriculture_forestry_and_fisheries',
 'animal_and_dairy_science',
 'veterinary_science',
 'agricultural_biotechnology',
 'other_agricultural_sciences',
 'psychology',
 'economics_and_business',
 'education',
 'sociology',
 'law',
 'political_science',
 'social_and_economic_geography',
 'media_and_communications',
 'other_social_sciences',
 'history_and_archaeology',
 'languages_and_literature',
 'philosophy_ethics_and_religion',
 'arts',
 'other_humanities_and_the_arts']